In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pathlib import Path
import math
import re

In [106]:
file = 'Figure_3b_data.xlsx'
excel_data = pd.read_excel(file)
pressure_data = pd.DataFrame(excel_data,columns=['Time(s).1','Pressure(Pa)']).dropna().values
exper_data = pd.DataFrame(excel_data,columns=['Time(s)','Lp/Rp']).dropna().values


In [107]:
pressure_change_index = np.where(pressure_data[1:,1] != pressure_data[:-1,1])[0]
pressure_change_index = np.append(0,pressure_change_index)
pressure_change_index = np.append(pressure_change_index,len(pressure_data)-1)
long_dp_index = np.where(pressure_change_index[1:] - pressure_change_index[:-1]>10)[0]
start_index = pressure_change_index[long_dp_index]+1
end_index = pressure_change_index[long_dp_index+1]+1
pressure_ref=pressure_data[start_index]
ref_pressure = 0
pressure_ref = np.append(0,pressure_ref[:,1]+ref_pressure)
delta_P=pressure_ref[1:]-pressure_ref[:-1]

In [108]:
file_path = Path('data_files/fig_3b')
Path.mkdir(file_path,exist_ok=True)
file_idx = 0
for i,j,dP in zip(start_index,end_index,delta_P):
    if dP:
        data_slice = exper_data[i+1:j,:]
        # header_txt = file + '\n Pressure (Pa):\n'+ str(pressure_data[i,1]) + '\n Delta Pressure (Pa):\n' + str(dP) + '\nt,L(t)'
        header_txt = 'file name\n' + file + '\npressure\n'+ "{:.1f}".format(pressure_data[i,1]) + '\ndelta pressure\n' + "{:.1f}".format(dP) + '\ncondensate\nRGG\nstate\nwetting'
        out_path = file_path/('seg_' + str(file_idx) + '.dat')
        file_idx = file_idx + 1
        np.savetxt(out_path,data_slice,delimiter=',',header=header_txt,fmt=['%d','%.3f'])

In [47]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx

In [99]:
file = 'oil-aspiration/102122-5.xlsx'
excel_data = pd.read_excel(file)
pressure_data = pd.DataFrame(excel_data,columns=['Time(s).1','Pressure(Pa)']).dropna().values
# pressure_data = pd.DataFrame(excel_data,columns=['Time.1','Pressure']).dropna().values

file_path = Path('data_files/oil_11')
Path.mkdir(file_path,exist_ok=True)
traj_data = excel_data.drop(columns=["Time(s)","Lp/Rp","Time(s).1","Pressure(Pa)"])
# traj_data = excel_data.drop(columns=['Time(s)','Pressure (Pa)'])
# traj_data = excel_data.drop(columns=["Time","Lp/Rp","Time.1","Pressure (Pa)"])
time_series_count = int(traj_data.columns.size/2)
pressure_ref = np.array([pressure_data[0,1]])
for i in range(0,time_series_count):
    time_data = np.array(traj_data.iloc[:,2*i].dropna().values)
    lp_data = np.array(traj_data.iloc[:,2*i+1].dropna().values)
    start_time = time_data[0]
    end_time = time_data[-1]

    pressure_start_idx = find_nearest(pressure_data[:,0],start_time)
    pressure_end_idx = find_nearest(pressure_data[:,0],end_time)
    avg_pressure = np.mean(pressure_data[pressure_start_idx:pressure_end_idx,1])
    dP = avg_pressure - pressure_ref[-1]
    pressure_ref = np.append(pressure_ref,np.array([avg_pressure]))
    data_slice = np.stack((time_data,lp_data),axis=1)
    # header_txt = 'file name\n' + file + '\npressure\n'+ "{:.1f}".format(avg_pressure) + '\ndelta pressure\n' + "{:.1f}".format(dP) + '\ncondensate\nSynapsin\nstate\nwetting'
    huan_comment = 'High Viscosity oil aspiration Pipette filled with ethanol'
    state = 'nonwetting'
    pipette_fill = 'ethanol'
    solvent = 'ethanol'
    lubricating_ethanol = 'false'
    header_txt = 'file name\n' + file + '\npressure\n'+ "{:.1f}".format(avg_pressure) + '\ndelta pressure\n' + "{:.1f}".format(dP) + '\nhuan comment\n' + huan_comment + '\nstate\n' + state + '\npipette fill\n' + pipette_fill + '\nsolvent\n'+solvent + '\nlubricating ethanol\n'+ lubricating_ethanol 
    out_path = file_path/('seg_' + str(i) + '.dat')
    np.savetxt(out_path,data_slice,delimiter=',',header=header_txt,fmt=['%.2f','%.3f'])




In [17]:
dp_times=[]
pattern = re.compile("Fitted linear_X(\d?\d?)")
for column in excel_data.columns:
    form_match = pattern.match(column)
    if form_match:
        dp_times.append(excel_data[column].iloc[0])